In [36]:
import math
import networkx as nx
import random
from datetime import datetime

In [4]:
G = nx.Graph()
graph = open("road-minnesota.txt","r")
# importance priority queue
imp_pq = list()
order = 0

In [5]:

def Connect(G, graph):
    # get number of nodes and edges
    graph_parameters = graph.readline().split()
    vertices_number = int(graph_parameters[0],base=10)
    edges_number = int(graph_parameters[1],base=10)
    print(vertices_number)
    print(edges_number)
    # add nodes to networkx graph
    for i in range(vertices_number):
        G.add_node(i+1, contracted = False,imp=0,level=0,contr_neighbours=0)
    # add edges to networkx graph
    edge = graph.readline()
    while edge:
        edge_parameters = edge.split()
        source_node = int(edge_parameters[0], base=10)
        target_node = int(edge_parameters[1], base=10)
        edge_weight = 1
        found_exist = False
        for i in G[source_node]:
            # already store the edge with different weight, choose the min weight
            if i == target_node:
                G[source_node][target_node]['weight'] = min(G[source_node][target_node]['weight'],edge_weight)
                found_exist = True
                break
        if not found_exist:
            G.add_edge(source_node,target_node,weight=edge_weight)
        edge = graph.readline()
    

In [6]:
def SetOrder(G, imp_pq, n):
    # initialize imp_pq
    for i in range(n):
        imp_pq.append((GetImportance(G,i+1),i+1))
        # print("settled node %d"%(i+1))
    order = 1
    # for i in range(1000):
    #     print(imp_pq[i+1])
    lazy_update_counter = 0
    contracted_num = 0
    completed_rate = 0
    k = 0
    print("initializing importance queue settled.")
    while len(imp_pq)>0:
        # find current lowest importance node in imp_pq
        curr_node_imp_pair = min(imp_pq, key= lambda pair:pair[0])
        curr_node  = curr_node_imp_pair[1]   
        imp_pq.remove(curr_node_imp_pair)
        # get new importance for current lowest importance node
        new_imp = GetImportance(G,curr_node)
        # print("get new importance for node %d"%curr_node)
        # lazy update
        if((len(imp_pq) == 0) or (new_imp - min(imp_pq,key=lambda pair:pair[0])[0] <= 10) or (lazy_update_counter >= 5)):
            # print("update for node %d"%curr_node)
            lazy_update_counter = 0
            G.nodes[curr_node]['imp'] = order
            order +=1
            # contract node
            G.nodes[curr_node]['contracted'] = True
            ContractNode(G,curr_node,n)
            # print("already contracted node %d"%curr_node)
            contracted_num += 1
            k +=1
            if(contracted_num == n):
                print("contracted completed!")
            if(k == 100):
                completed_rate = contracted_num / n
                print(completed_rate)
                k = 0
        else:
            imp_pq.append((new_imp,curr_node))
            lazy_update_counter +=1
            # print("recalculated prority of node %d" %curr_node)
            

In [7]:
def GetImportance(G, x):
    # get number of incident edges of x
    edges_incident = len(G[x])
    # get number of added shortcut when simulate contracting node x
    shortcuts = 0
    seenBefore = list()
    for i in G[x]:
        for j in G[x]:
            pair = sorted((i,j))
            if (i==j or (pair in seenBefore)):continue
            seenBefore.append(pair)
            if((G.nodes[i]['contracted'] == False) and (G.nodes[j]['contracted'] == False)):
                shortcuts +=1
    edge_difference = shortcuts - edges_incident
    return edge_difference + 2*G.nodes[x]['contr_neighbours'] + G.nodes[x]['level']

In [8]:
def ContractNode(G, x, n):
    mx = GetMaxEdge(G, x)
    seenBefore = list()
    for i in G[x]:
        for j in G[x]:
            if ((G.nodes[i]['contracted'] == True) or (G.nodes[j]['contracted'])):
                continue
            pair = sorted((i,j))
            if (i==j or (pair in seenBefore)):continue
            seenBefore.append(pair)
            Check_Witness(G, n, i, x, mx)
            # print("check witness completed")
    # update importance term in incident node
    for i in G[x]:
        G.nodes[i]['contr_neighbours'] +=1
        G.nodes[i]['level'] = max(G.nodes[i]['level'], G.nodes[x]['level'] + 1)
            

In [9]:
def GetMaxEdge(G, x):
    ret = 0
    for i in G[x]:
        for j in G[x]:
            if((i != j) and (G.nodes[i]['contracted'] == False) and (G.nodes[j]['contracted'] == False)):
                ret = max(ret, G[x][i]['weight'] + G[x][j]['weight'])
    return ret

In [10]:
def Check_Witness(G, n, u, x, mx, type=None):
    # dijkstra priority queue for search witness path from u to v, excludes x
    # v is incident edge of x, excludes u
    D_pq = list()
    # initialize D_pq
    D_pq.append((0, u))
    # distance dictionary from u to any node in search tree
    D_dist = dict()
    # initialize D_dist
    D_dist[u] = 0
    # maximum iteration round for dijkstra search
    iter = int(250 * (n - order) / n)
    while((len(D_pq) > 0) and (iter > 0)):
        iter -=1
        curr_dist_pair = min(D_pq, key= lambda pair:pair[0])
        curr_dist = curr_dist_pair[0]
        a = curr_dist_pair[1]
        D_pq.remove(curr_dist_pair)
        if(curr_dist > D_dist[a]):
            continue
        for p in G[a]:
            new_dist = curr_dist + G[a][p]['weight']
            # p must not be x and not be contracted
            if(p != x and (G.nodes[p]['contracted'] == False)):
                # p must not be settled node or distance greater than new_dist
                if((p not in D_dist) or (D_dist[p] > new_dist)):
                    # prune when witness path greater than mx
                    if(p not in D_dist):
                        if new_dist < mx:
                            D_dist[p] = new_dist
                            D_pq.append((new_dist,p))
                    else:
                        if(D_dist[p] < mx):
                            D_dist[p] = new_dist
                            D_pq.append((new_dist,p))
    for v in G[x]:
        # v can not be u and not be contracted
        if ((v!=u) and (G.nodes[v]['contracted'] == False)):
            new_w = G[u][x]['weight'] + G[x][v]['weight']
            # print("%d %d %d"%(u,v,new_w))
            if((v not in D_dist) or (D_dist[v] > new_w)):
                # add shortcut
                # try:
                #     if(u,v) in G.edges:
                #         print("run here: no more add_edge")
                #         continue
                # except:
                G.add_edge(u,v,weight=new_w)
                # print("run here: add_edge:%d %d"%(u,v))
            

In [11]:
def GetDistance(G, s, t):
    # search with bi-dijkstra with ordering rank
    # initializing dijkstra from source node s
    SP_s = dict()
    parent_s = dict()
    unrelaxed_s = list()
    for node in G.nodes():
        SP_s[node] = math.inf
        parent_s[node] = None
        unrelaxed_s.append(node)
    SP_s[s] = 0
    # dijkstra forward
    while unrelaxed_s:
        node = min(unrelaxed_s, key= lambda node:SP_s[node])
        unrelaxed_s.remove(node)
        if SP_s[node] == math.inf:
            break
        # G[node] are the incident edges of node
        for child in G[node]:
            # skip unqualified edges
            if G.nodes[child]['imp'] < G.nodes[node]['imp']:
                continue
            distance = SP_s[node] + G[node][child]['weight']
            # relax edge
            if distance < SP_s[child]:
                SP_s[child] = distance
                parent_s[child] = node
    # initializing dijkstra from target node t
    SP_t = dict()
    parent_t = dict()
    unrelaxed_t = list()
    for node in G.nodes():
        SP_t[node] = math.inf
        parent_t[node] = None
        unrelaxed_t.append(node)
    SP_t[t] = 0

    # dijkstra backward
    while unrelaxed_t:
        node = min(unrelaxed_t, key= lambda node: SP_t[node])
        unrelaxed_t.remove(node)
        if SP_t[node] == math.inf:
            break
        # G[node] are the incident edges of node
        for child in G[node]:
            # skip unqualified edges
            if G.nodes[child]['imp'] < G.nodes[node]['imp']:
                continue
            distance = SP_t[node] + G[node][child]['weight']
            if distance < SP_t[child]:
                SP_t[child] = distance
                parent_t[child] = node
    minimum = math.inf
    merge_node = None
    for i in SP_s:
        if SP_t[i] == math.inf:
            continue
        if SP_t[i] + SP_s[i] < minimum:
            minimum = SP_s[i] + SP_t[i]
            merge_node = i
    return minimum, merge_node, SP_s, SP_t, parent_s, parent_t

In [12]:
# see the route from origin of dijkstra to a given node
def Route_dijkstra(parent, node):
    route = []
    while node != None:
        route.append(node)
        node = parent[node]
    return route[::-1]

In [13]:
def See_full_route(G, s, t):
    minimum, merge_node, SP_s, SP_t, parent_s, parent_t = GetDistance(G, s, t)
    print("shortest distance between source node %d to target node %d:"%(s,t))
    if minimum == math.inf:
        print("no path between source node %d to target node %d"%(s,t))
        return
    has_path_list.append([s,t])
    print(minimum)
    route_from_source = Route_dijkstra(parent_s, merge_node)
    # show route
    print("route from source node %d:"%s)
    print(route_from_source)
    route_from_target = Route_dijkstra(parent_t, merge_node)
    # show route
    print("route from target node %d:"%t)
    print(route_from_target)
    route = route_from_source + route_from_target[::-1][1:]
    # show route
    print("entire route:")
    print(route)
    unvisited = 0
    for s_node, s_dist in SP_s.items():
        for t_node, t_dist in SP_t.items():
            if s_node == t_node and s_dist == t_dist == math.inf:
                unvisited += 1
    print(f"""we have skipped {unvisited} nodes from a graph with {len(G)}, 
    so we have skipped {unvisited/len(G)*100}% of the nodes in our search space.""")
    print("\n\n")

In [14]:
Connect(G, graph)

2642
3303


In [15]:
edges_before = [*G.edges()]

In [16]:
# G.nodes.data()
for i in range(100):
    print(i+1,G.nodes[i+1])

1 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
2 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
3 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
4 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
5 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
6 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
7 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
8 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
9 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
10 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
11 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
12 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
13 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
14 {'contracted': False, 'imp': 0, 'level': 0, 'contr_neighbours': 0}
15 {'contracted': False, 'imp

In [17]:
SetOrder(G, imp_pq, len(G.nodes))

initializing importance queue settled.
0.03785011355034065
0.0757002271006813
0.11355034065102196
0.1514004542013626
0.18925056775170326
0.22710068130204392
0.26495079485238454
0.3028009084027252
0.34065102195306585
0.3785011355034065
0.41635124905374715
0.45420136260408783
0.49205147615442846
0.5299015897047691
0.5677517032551098
0.6056018168054504
0.6434519303557911
0.6813020439061317
0.7191521574564723
0.757002271006813
0.7948523845571537
0.8327024981074943
0.8705526116578349
0.9084027252081757
0.9462528387585163
0.9841029523088569
contracted completed!


In [18]:
edges_after = [*G.edges()]
print("# edges before", len(edges_before))
print("# edges after", len(edges_after))

# edges before 3303
# edges after 7453


In [19]:
added_edges = list(set(edges_after) - set(edges_before))
added_edges

[(690, 855),
 (2450, 2487),
 (2134, 2297),
 (1745, 1797),
 (913, 934),
 (2293, 2336),
 (1656, 1766),
 (2087, 2118),
 (1608, 1728),
 (2069, 2076),
 (1867, 1917),
 (48, 86),
 (104, 115),
 (945, 984),
 (341, 874),
 (1131, 1143),
 (1813, 1867),
 (490, 506),
 (2110, 2259),
 (702, 728),
 (153, 202),
 (2491, 2519),
 (2188, 2491),
 (1301, 1435),
 (196, 229),
 (1762, 1951),
 (1020, 1034),
 (806, 874),
 (589, 815),
 (326, 458),
 (1346, 1367),
 (1419, 1463),
 (560, 571),
 (1470, 1610),
 (137, 164),
 (1315, 1382),
 (2347, 2534),
 (606, 742),
 (19, 639),
 (1570, 1756),
 (391, 411),
 (2450, 2478),
 (582, 704),
 (470, 549),
 (788, 861),
 (2110, 2123),
 (1451, 1466),
 (540, 544),
 (1929, 2079),
 (119, 269),
 (1476, 1483),
 (1171, 1183),
 (1744, 1766),
 (2004, 2188),
 (2131, 2227),
 (274, 281),
 (129, 140),
 (1826, 1951),
 (1933, 2009),
 (457, 527),
 (1435, 1554),
 (202, 305),
 (1134, 1624),
 (1241, 1831),
 (2245, 2298),
 (1390, 1408),
 (1360, 1390),
 (898, 933),
 (1836, 1847),
 (150, 160),
 (560, 562)

In [20]:
query_list = list()
i = 0
while i < 100:
    a = random.randint(1,len(G.nodes))
    b = random.randint(1,len(G.nodes))
    pair = sorted((a,b))
    if (a==b or (pair in query_list)):
        continue
    query_list.append(pair)
    i +=1
query_list

[[442, 1000],
 [739, 2234],
 [589, 1495],
 [531, 1175],
 [1359, 1363],
 [1215, 2559],
 [1467, 2444],
 [1021, 1584],
 [1416, 2083],
 [636, 1159],
 [1970, 2369],
 [487, 1500],
 [293, 1879],
 [679, 2141],
 [91, 1780],
 [2518, 2568],
 [772, 1592],
 [877, 923],
 [1083, 1616],
 [1848, 1926],
 [886, 1279],
 [378, 1162],
 [957, 1796],
 [1412, 1590],
 [510, 1435],
 [377, 2332],
 [833, 1792],
 [507, 1317],
 [445, 2101],
 [390, 1084],
 [1045, 1743],
 [993, 1785],
 [2398, 2559],
 [644, 1481],
 [590, 2015],
 [35, 41],
 [1368, 2046],
 [1173, 2226],
 [892, 2539],
 [564, 1183],
 [2131, 2464],
 [719, 1306],
 [79, 2636],
 [1067, 1924],
 [2323, 2472],
 [346, 1207],
 [160, 1719],
 [900, 2321],
 [824, 1654],
 [14, 1602],
 [825, 1073],
 [831, 1373],
 [40, 1339],
 [1926, 2138],
 [552, 2470],
 [302, 893],
 [1662, 2432],
 [74, 2373],
 [1524, 2442],
 [1034, 1726],
 [474, 2554],
 [978, 1575],
 [365, 890],
 [17, 619],
 [1099, 1139],
 [1246, 2314],
 [1316, 2546],
 [1919, 2331],
 [923, 2356],
 [1897, 2230],
 [625, 

In [23]:
for i in range(100):
    See_full_route(G,query_list[i][0],query_list[i][1])

shortest distance between source node 442 to target node 1000:
35
route from source node 442:
[442, 414, 874]
route from target node 1000:
[1000, 1001, 945, 874]
entire route:
[442, 414, 874, 945, 1001, 1000]
we have skipped 2580 nodes from a graph with 2642, 
    so we have skipped 97.65329295987888% of the nodes in our search space.



shortest distance between source node 739 to target node 2234:
57
route from source node 739:
[739, 742, 976, 1782]
route from target node 2234:
[2234, 2262, 2245, 2071, 2110, 2099, 2137, 1782]
entire route:
[739, 742, 976, 1782, 2137, 2099, 2110, 2071, 2245, 2262, 2234]
we have skipped 2568 nodes from a graph with 2642, 
    so we have skipped 97.19909159727479% of the nodes in our search space.



shortest distance between source node 589 to target node 1495:
27
route from source node 589:
[589, 828]
route from target node 1495:
[1495, 1270, 1272, 1024, 828]
entire route:
[589, 828, 1024, 1272, 1270, 1495]
we have skipped 2590 nodes from a graph with

In [22]:
G_original = nx.Graph()
graph = open("road-euroroad.txt","r")
Connect(G_original,graph)
has_path_list = list()

1174
1417


In [35]:
from networkx.algorithms.shortest_paths.weighted import single_source_dijkstra
single_source_dijkstra(G_original, 442, 1000)
# See_full_route(G,442,1001)

(17,
 [442,
  441,
  440,
  439,
  438,
  437,
  436,
  321,
  320,
  866,
  297,
  284,
  277,
  378,
  377,
  998,
  999,
  1000])

In [34]:
for i in range(100):
    print(i+1,G.nodes[i+1])

1 {'contracted': True, 'imp': 1, 'level': 2, 'contr_neighbours': 1}
2 {'contracted': True, 'imp': 2, 'level': 5, 'contr_neighbours': 1}
3 {'contracted': True, 'imp': 3, 'level': 5, 'contr_neighbours': 2}
4 {'contracted': True, 'imp': 4, 'level': 5, 'contr_neighbours': 2}
5 {'contracted': True, 'imp': 5, 'level': 3, 'contr_neighbours': 1}
6 {'contracted': True, 'imp': 6, 'level': 2, 'contr_neighbours': 1}
7 {'contracted': True, 'imp': 1536, 'level': 3, 'contr_neighbours': 3}
8 {'contracted': True, 'imp': 7, 'level': 2, 'contr_neighbours': 1}
9 {'contracted': True, 'imp': 1537, 'level': 21, 'contr_neighbours': 4}
10 {'contracted': True, 'imp': 1538, 'level': 21, 'contr_neighbours': 4}
11 {'contracted': True, 'imp': 1539, 'level': 21, 'contr_neighbours': 5}
12 {'contracted': True, 'imp': 1540, 'level': 21, 'contr_neighbours': 6}
13 {'contracted': True, 'imp': 8, 'level': 21, 'contr_neighbours': 2}
14 {'contracted': True, 'imp': 9, 'level': 3, 'contr_neighbours': 1}
15 {'contracted': True,

In [37]:
def See_full_route_constract(G, s, t):
    minimum, merge_node, SP_s, SP_t, parent_s, parent_t = GetDistance(G, s, t)
    # print("shortest distance between source node %d to target node %d:"%(s,t))
    if minimum == math.inf:
        print("no path between source node %d to target node %d"%(s,t))
        return
    # has_path_list.append([s,t])
    # print(minimum)
    route_from_source = Route_dijkstra(parent_s, merge_node)
    # show route
    # print("route from source node %d:"%s)
    # print(route_from_source)
    route_from_target = Route_dijkstra(parent_t, merge_node)
    # show route
    # print("route from target node %d:"%t)
    # print(route_from_target)
    route = route_from_source + route_from_target[::-1][1:]
    # show route
    # print("entire route:")
    # print(route)
    # unvisited = 0
    # for s_node, s_dist in SP_s.items():
    #     for t_node, t_dist in SP_t.items():
    #         if s_node == t_node and s_dist == t_dist == math.inf:
    #             unvisited += 1
    # print(f"""we have skipped {unvisited} nodes from a graph with {len(G)}, 
    # so we have skipped {unvisited/len(G)*100}% of the nodes in our search space.""")
    # print("\n\n")
    # print([minimum, route])

In [38]:
def dijkstra_with_contraction(G, source, destination, contracted = None):
    nx.set_node_attributes(G, {contracted: True}, 'contracted')
    
        
    shortest_path = dict()
    heap = list()
    
    for i in G.nodes():
        if not nx.get_node_attributes(G, 'contracted')[i]:
            shortest_path[i] = math.inf
            heap.append(i)
    shortest_path[source] = 0
    
    while len(heap)>0:
        q = min(heap, key= lambda node : shortest_path[node])
        if q == destination:
            nx.set_node_attributes(G, {contracted: False}, 'contracted')
            return shortest_path[q]
        heap.remove(q)
        # G[q] are incident edges of q
        for v in G[q]:
            # if the node is contracted, skip it
            if not nx.get_node_attributes(G, 'contracted')[v]:
                distance = shortest_path[q] + G[q][v]['weight']
                if distance < shortest_path[v]:
                    shortest_path[v] = distance
    nx.set_node_attributes(G, {contracted: False}, 'contracted')
    
    # can not reach the destination
    return math.inf

In [44]:
from networkx.algorithms.shortest_paths.weighted import single_source_dijkstra
sd_a = datetime.now()
for i in range(len(query_list)):
    distance = dijkstra_with_contraction(G_original, query_list[i][0], query_list[i][1])
sd_b = datetime.now()
((sd_b - sd_a).total_seconds())/len(query_list)

1.90200824

In [43]:
ch_a = datetime.now()
for i in range(len(query_list)):
    See_full_route_constract(G, query_list[i][0], query_list[i][1])
ch_b = datetime.now()
((ch_b - ch_a).total_seconds())/len(query_list)

0.03836821